In [1]:
import sys
import os
import glob
import json
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np

In [2]:
def find_files(path):
    return glob.glob(path)

In [3]:
def read_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

### Adat beolvasása JSON fájlból

In [4]:
json_data = []

for file in find_files('Data/*'):
    print("Reading file:", file)
    json_data.append(read_json(file))

Reading file: Data/gyakori_szamitastechnika
Reading file: Data/gyakori_egeszseg_20000
Reading file: Data/gyakori_szorakozas_30000
Reading file: Data/gyakori_egeszseg
Reading file: Data/gyakori_allatok_14000


### Dataframe-mé alakítás

In [5]:
data_frames = []

for data in json_data:
    data_frames.append(pd.DataFrame(data))

### Bepillantás a kérdésekbe

In [6]:
data_frames[0].head(2)

,valasz,kategoriak,hosszu_kerdes,rovid_kerdes,keywords
0,Várak régen is voltak. Ha mindhárom tornyot le...,"[Számítástechnika, Internet]",Miért lett ilyen sz@r a honfoglaló? Régen tök ...,Miért lett ilyen sz@r a honfoglaló?,"[Honfoglaló, vár]"
1,"Ahogy írták, az stdio az a C-s függvénykönyvtá...","[Számítástechnika, Programozás]",C++-ban melyiket érdemesebb használni? Stdio v...,C++-ban melyiket érdemesebb használni? Stdio v...,"[C++, iostream, konzol, Windows, Visual Studio]"


### Kérdések megszámolása, átlagos hossz számolása

In [7]:
def calculate_avg(data_frames, label="rovid_kerdes"):
    avg = 0
    count = 0
    
    for frame in data_frames:
        for index, row in frame.iterrows():
            avg += len(row[label])
            count += 1

    avg = avg / count
    
    return avg, count

### Extrém rövid/hosszú kérdések eldobása

In [8]:
def drop_extreme(data_frames, min_, max_, label="rovid_kerdes"):
    for idx, frame in enumerate(data_frames):
        frame = frame[frame[label].map(len) >= min_]
        frame = frame[frame[label].map(len) <= max_]

        data_frames[idx] = frame

In [9]:
long_questions_avg_len, questions_count = calculate_avg(data_frames, "hosszu_kerdes")

print("Number of questions: %d" % questions_count)
print("Average length: %f" % long_questions_avg_len)

Number of questions: 168127
Average length: 346.495816


In [10]:
drop_extreme(data_frames, long_questions_avg_len / 4, long_questions_avg_len * 3, "hosszu_kerdes")
long_questions_avg_len, questions_count = calculate_avg(data_frames, "hosszu_kerdes")

print("Number of questions: %d" % questions_count)
print("Average length: %f" % long_questions_avg_len)

Number of questions: 137265
Average length: 326.753950


In [11]:
target_dir = {}
target_names = []

target_values = []
long_questions = []

for frame in data_frames:
    for idx in frame.index:
        category = frame["kategoriak"][idx][0]
        if category not in target_names:
            target_dir[category] = len(target_dir)
            target_names.append(category)
        target_values.append(target_dir[category])
        long_questions.append(frame["hosszu_kerdes"][idx])
print(target_names)        

['Számítástechnika', 'Egészség', 'Szórakozás', 'Állatok']


In [12]:
import random

paired = list(zip(target_values, long_questions))

random.shuffle(paired)

target_values, long_questions = zip(*paired)
long_questions = list(long_questions)

for i in range(0, 3):
    print(target_names[target_values[i]])
    print(long_questions[i])
    print("--------------------------------------")

Állatok
Hová tűnnek a garnélák petéi? Már a második garnélámmal jártam úgy, hogy kb 3-4 héttel a petésedés után egyik napról a másikra eltűntek a peték a hasáról. Már szépen látszottak a kis pontok amik a kis garnélák szemei voltak a petéken, és az elsőnél azt hittem hogy kikeltek, de egyet sem látok az akváriumon most a második nőstény pete nélkülivé válása utána is.Ez az első alkalom hogy szaporodni kezdtek nálam, és nem tudom, lehet kikeltek de olyan kicsik hogy nem látom őket? Kerestem de egyet se láttam, az a baj hogy nem tudom mit kéne keressek. Lehet hogy a halivadékok megették őket? Egy centinél kisebb ivadékok vannak csak, amint elérik az 1 cm-t kiveszem őket a nagy akváriumba. Ők csak nem ehették meg őket, nem?
--------------------------------------
Szórakozás
Hogy indíthatok pert, egy MMORPG privát szerver ellen? Hol tudnék utána nézni játék jogtörvényeknek? Lehetne ezt interneten keresztül kikeresni? Elérkeztem arra a pontra, hogy egy MMORPG szerver ellen belefogjak egy per

In [13]:
def unicode_to_ascii(data):
    return " ".join(unidecode.unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", data).lower()).split())

In [14]:
import unidecode
import re
from collections import Counter

vocab = Counter()

for idx, q in enumerate(long_questions):
    words = unidecode.unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", q).lower()).split()
    for word in words:
        vocab[word] += 1
    long_questions[idx] = " ".join(words)

total_words = len(vocab)

word_to_index = {}

for idx, word in enumerate(vocab):
    word_to_index[word] = idx

In [15]:
def get_batch(text, target, i, batch_size):

    batches = []
    results = []
    
    texts = text[i * batch_size : i * batch_size + batch_size]
    categories = target[i * batch_size : i * batch_size + batch_size]

    for text in texts:
        layer = np.zeros(total_words , dtype=float)
        for word in text.split():
            layer[word_to_index[word.lower()]] += 1
            
        batches.append(layer)
        
    for category in categories:
        results.append(category)
     
    return np.array(batches), np.array(results)

In [16]:
learning_rate = 0.01
num_epochs = 3
batch_size = 150
display_step = 1

hidden_size = 100
input_size = total_words
num_classes = len(target_names)

In [17]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [18]:
class OurNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
 
     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [19]:
loss = nn.CrossEntropyLoss()
input_ = Variable(torch.randn(2, 5), requires_grad=True)
target = Variable(torch.LongTensor(2).random_(5))
output = loss(input_, target)
output.backward()

In [26]:
net = OurNet(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

train_text = long_questions[:6000]
train_target = target_values[:6000]

for epoch in range(num_epochs):
    total_batch = int(len(train_text) / batch_size)

    for i in range(total_batch):
        batch_x,batch_y = get_batch(train_text, train_target, i, batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        
        optimizer.zero_grad()
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %(epoch + 1, num_epochs, i + 1, len(train_text) // batch_size, loss.data))

Epoch [1/5], Step [4/40], Loss: 1.2716
Epoch [1/5], Step [8/40], Loss: 0.9863
Epoch [1/5], Step [12/40], Loss: 0.7888
Epoch [1/5], Step [16/40], Loss: 0.6884
Epoch [1/5], Step [20/40], Loss: 0.5506
Epoch [1/5], Step [24/40], Loss: 0.7647
Epoch [1/5], Step [28/40], Loss: 0.5964
Epoch [1/5], Step [32/40], Loss: 0.6129
Epoch [1/5], Step [36/40], Loss: 0.4186
Epoch [1/5], Step [40/40], Loss: 0.5360
Epoch [2/5], Step [4/40], Loss: 0.0890
Epoch [2/5], Step [8/40], Loss: 0.0830
Epoch [2/5], Step [12/40], Loss: 0.0697
Epoch [2/5], Step [16/40], Loss: 0.0481
Epoch [2/5], Step [20/40], Loss: 0.0098
Epoch [2/5], Step [24/40], Loss: 0.0510
Epoch [2/5], Step [28/40], Loss: 0.0299
Epoch [2/5], Step [32/40], Loss: 0.0545
Epoch [2/5], Step [36/40], Loss: 0.0105
Epoch [2/5], Step [40/40], Loss: 0.0209
Epoch [3/5], Step [4/40], Loss: 0.0074
Epoch [3/5], Step [8/40], Loss: 0.0008
Epoch [3/5], Step [12/40], Loss: 0.0005
Epoch [3/5], Step [16/40], Loss: 0.0005
Epoch [3/5], Step [20/40], Loss: 0.0003
Epoch 

In [ ]:
print("Mennyi kérdésed van?")
q_num = int(input())

for q in range(0, q_num):
    test_text = input()

    test_data = [unicode_to_ascii(test_text)]
    total_test_data = 1

    batch_x_test, batch_y_test = get_batch(test_data, [0], 0, total_test_data)
    articles = Variable(torch.FloatTensor(batch_x_test))
    labels = torch.LongTensor(batch_y_test)
    outputs = net(articles)
    _, predicted = torch.max(outputs.data, 1)

    print("A kérdés %s témájú .. talán." % target_names[predicted.item()])

Mennyi kérdésed van?
3
